In [28]:
import os
%pwd

'd:\\'

In [29]:
os.chdir("D:\Image_Classification_CNN_and_MLFlow")

In [30]:
%pwd

'D:\\Image_Classification_CNN_and_MLFlow'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weight : str
    params_classes: int

In [25]:
from src.CNNClassifier.constants import *
from src.CNNClassifier.utils.common import read_yaml,create_directories

In [32]:
class ConfigurationManager1:
    def __init__(self,
                #  config_filePath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(Path("config/config.yaml"))
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config=self.config.prepare_base_model
        params = self.params
        create_directories([config.root_dir])

        base_model_config=PrepareBaseModelConfig(
                root_dir = config.root_dir,
                base_model_path = config.base_model_path,
                updated_base_model_path = config.updated_base_model_path,
                params_image_size = params.IMAGE_SIZE,
                params_learning_rate = params.LEARNING_RATE,
                params_include_top = params.INCLUDE_TOP,
                params_weight = params.WEIGHTS,
                params_classes= params.CLASSES,
            
        )
        return base_model_config



In [12]:
import tensorflow as tf


In [35]:

class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            include_top=self.config.params_include_top

        )

        self.save_model(path=self.config.base_model_path,model=self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):

        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    

    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)



In [36]:
try:
    config=ConfigurationManager1()
    base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e



[2024-03-18 20:35:03,803: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-03-18 20:35:03,810: INFO:common:yaml file: params.yaml loaded successfully]
[2024-03-18 20:35:03,814: INFO:common:created directory at: artifacts]
[2024-03-18 20:35:03,816: INFO:common:created directory at: artifacts/prepare_base_model]


[2024-03-18 20:35:12,294: WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                            